# Vehicle Detection with BG_SubtractorMOG2: (Research Part)

In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
# load a Video:
cap = cv2.VideoCapture('C:/Users/subha/Desktop/Research Contracts!/Vehicle Detection & Tracking!/VDT!/Car-1!.mp4')

# You can optionally work on the live web cam
# cap = cv2.VideoCapture(0)

# Create the Background Object, you can choose to detect shadows or not: (if True they will be shown as gray)
backgroundobject = cv2.createBackgroundSubtractorMOG2( history = 2, detectShadows = True )

while(1):
    ret, frame = cap.read()  
    if not ret:
        break
        
    # Apply the background object on each frame:
    fgmask = backgroundobject.apply(frame)

    # Also extracting the real detected foreground part of the image (optional):
    real_part = cv2.bitwise_and(frame,frame,mask=fgmask)
    
    # Making fgmask 3 channeled so it can be stacked with others:
    fgmask_3 = cv2.cvtColor(fgmask, cv2.COLOR_GRAY2BGR)
    
    # Stack all three frames and show the image:
    stacked = np.hstack((fgmask_3, frame, real_part))
    cv2.imshow('Sample',cv2.resize(stacked, None, fx=0.65, fy=0.65))
 
    k = cv2.waitKey(30) &  0xff
    if k == 27:
        break
   
cap.release()
cv2.destroyAllWindows()

In [3]:
# load a video:
video = cv2.VideoCapture('C:/Users/subha/Desktop/Research Contracts!/Vehicle Detection & Tracking!/VDT!/Car-1!.mp4')

# You can set custom kernel size if you want:
kernel = None

# Initialize the background object:
backgroundObject = cv2.createBackgroundSubtractorMOG2(detectShadows = False)

while True:
    
    # Read a new frame:
    ret, frame = video.read()

    # Check if frame is not read correctly:
    if not ret:
        
        # Break the loop:

        break

    # Apply the background object on the frame to get the segmented mask: 
    fgmask = backgroundObject.apply(frame)
    #initialMask = fgmask.copy()
    
    # Perform thresholding to get rid of the shadows:
    _, fgmask = cv2.threshold(fgmask, 250, 255, cv2.THRESH_BINARY)
    #noisymask = fgmask.copy()
    
    # Apply some morphological operations to make sure you have a good mask:
    fgmask = cv2.erode(fgmask, kernel, iterations = 1)
    fgmask = cv2.dilate(fgmask, kernel, iterations = 2)
    
    # Detect contours in the frame:
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create a copy of the frame to draw bounding boxes around the detected cars:
    frameCopy = frame.copy()
    
    # loop over each contour found in the frame:
    for cnt in contours:
        
        # Make sure the contour area is somewhat higher than some threshold to make sure its a car and not some noise:
        if cv2.contourArea(cnt) > 400:
            
            # Retrieve the bounding box coordinates from the contour:
            x, y, width, height = cv2.boundingRect(cnt)
            
            # Draw a bounding box around the Bike:
            cv2.rectangle(frameCopy, (x , y), (x + width, y + height),(0, 0, 255), 2)
            
            # Write Bike Detected near the bounding box drawn:
            cv2.putText(frameCopy, 'Car Detected', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,0), 1, cv2.LINE_AA)
    
    # Extract the foreground from the frame using the segmented mask:
    foregroundPart = cv2.bitwise_and(frame, frame, mask=fgmask)
        
    # Stack the original frame, extracted foreground, and annotated frame:
    stacked = np.hstack((frame, foregroundPart))

    # Display the stacked image with an appropriate title.
    cv2.imshow('Original Frame, Extracted Foreground and Detected Cars', cv2.resize(stacked, None, fx=0.5, fy=0.5))
    #cv2.imshow('initial Mask', initialMask)
    #cv2.imshow('Noisy Mask', noisymask)
    #cv2.imshow('Clean Mask', fgmask)


    # Wait until a key is pressed.
    # Retreive the ASCII code of the key pressed
    k = cv2.waitKey(1) & 0xff
    
    # Check if 'q' key is pressed.
    if k == ord('q'):
        
        # Break the loop.
        break

# Release the VideoCapture Object.
video.release()

# Close the windows.q
cv2.destroyAllWindows()

# Vehicle Counter with Background_SubtractorMOG2: (Project Part)

In [5]:
pip install opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.


In [9]:
import cv2
import numpy as np

# Load Video:
cap = cv2.VideoCapture('C:/Users/subha/Desktop/Research Contracts!/Vehicle Detection & Tracking!/VDT!/Car-1!.mp4')

# Declaring Rectangle Dimensions:

min_width_react = 80
min_height_react = 80
count_line_position = 550

# Innitialize Subtractor:

algo = cv2.createBackgroundSubtractorMOG2()

def center_handle(x,y,w,h):
    x1=int(w/2)
    y1=int(h/2)
    cx = x+x1
    cy = y+y1
    return cx, cy
detect = []
offset = 5 # Allowable Error Between Pixels
counter = 0

while True:
    ret, frame1 = cap.read()
    grey = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey, (3, 3), 5)
    
    # Applying on each Frame:
    img_sub = algo.apply(blur)
    dilat = cv2.dilate(img_sub, np.ones((1,1)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1,1))
    dilatada = cv2.morphologyEx(dilat, cv2.MORPH_CLOSE, kernel)
    dilatada = cv2.morphologyEx(dilatada, cv2.MORPH_CLOSE, kernel)
    contourShape,h = cv2.findContours(dilatada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    cv2.line(frame1, (25, count_line_position),(1200, count_line_position), (255,127,0), 3)
    
    
    for (i,c) in enumerate(contourShape):
        (x,y,w,h) = cv2.boundingRect(c)
        validate_counter = (w>= min_width_react) and (h>= min_height_react)
        if not validate_counter:
            continue
            
        cv2.rectangle(frame1, (x,y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(frame1, "Vehicle"+str(counter), (x, y - 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,244,255),5)
        
        center = center_handle(x,y,w,h)
        detect.append(center)
        cv2.circle(frame1, center, 4, (0,0,255), -1)
        
        for (x,y) in detect:
            if y<(count_line_position+offset) and y>(count_line_position-offset):
                counter+=1
                cv2.line(frame1, (25, count_line_position),(1200, count_line_position), (0,127,255), 3)
                detect.remove((x,y))
                print("Vehicle Counter:"+str(counter))
        
    cv2.putText(frame1, "Vehicle Counter:"+str(counter), (450, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255),5)
            
    
#     cv2.imshow('Detector', dilatada)
    
    
    cv2.imshow('Original Video', frame1)
    
    
    # Retreive the ASCII code of the key pressed
    k = cv2.waitKey(1) & 0xff
    
    # Check if 'q' key is pressed.
    if k == ord('q'):
        
        # Break the loop.
        break

# Close the windows.q
cv2.destroyAllWindows()

Vehicle Counter:1
Vehicle Counter:2
Vehicle Counter:3
Vehicle Counter:4
Vehicle Counter:5
Vehicle Counter:6
Vehicle Counter:7
Vehicle Counter:8
Vehicle Counter:9
Vehicle Counter:10
Vehicle Counter:11
Vehicle Counter:12
Vehicle Counter:13
Vehicle Counter:14


# Final:

In [10]:
import cv2
import numpy as np
import time

# Load Video:
cap = cv2.VideoCapture('C:/Users/subha/Desktop/Research Contracts!/Vehicle Detection & Tracking!/VDT!/Car-1!.mp4')
fps_start_time = 0
fps = 0

# Declaring Rectangle Dimensions:
min_width_react = 80
min_height_react = 80
count_line_position = 550

# Innitialize Subtractor:
algo = cv2.createBackgroundSubtractorMOG2()

def center_handle(x,y,w,h):
    x1=int(w/2)
    y1=int(h/2)
    cx = x+x1
    cy = y+y1
    return cx, cy
detect = []
offset = 5 # Allowable Error Between Pixels
counter = 0

while True:
    ret, frame1 = cap.read()
    
    # FPS Calculator:
    fps_end_time = time.time()
    time_diff = fps_end_time - fps_start_time
    fps = 1/(time_diff)
    fps_start_time = fps_end_time
    fps_text = "FPS: {:.2f}".format(fps)
    cv2.putText(frame1, fps_text, (5, 30), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,0),2)
    
    
    grey = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey, (3, 3), 5)
    
    # Applying on each Frame:
    img_sub = algo.apply(blur)
    dilat = cv2.dilate(img_sub, np.ones((1,1)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1,1))
    dilatada = cv2.morphologyEx(dilat, cv2.MORPH_CLOSE, kernel)
    dilatada = cv2.morphologyEx(dilatada, cv2.MORPH_CLOSE, kernel)
    contourShape,h = cv2.findContours(dilatada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    cv2.line(frame1, (25, count_line_position),(1200, count_line_position), (0,0,0), 2)
    
    # Vehicle Counter:
    for (i,c) in enumerate(contourShape):
        (x,y,w,h) = cv2.boundingRect(c)
        validate_counter = (w>= min_width_react) and (h>= min_height_react)
        if not validate_counter:
            continue
            
        cv2.rectangle(frame1, (x,y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(frame1, "Vehicle"+str(counter), (x, y - 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2)
        
        center = center_handle(x,y,w,h)
        detect.append(center)
        cv2.circle(frame1, center, 4, (0,0,255), -1)
        
        for (x,y) in detect:
            if y<(count_line_position+offset) and y>(count_line_position-offset):
                counter+=1
                cv2.line(frame1, (25, count_line_position),(1200, count_line_position), (0,0,0), 2)
                detect.remove((x,y))
                print("Vehicle Counter:"+str(counter))
        
    cv2.putText(frame1, "Vehicle Counter:"+str(counter), (450, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0),3)
            
    
#     cv2.imshow('Detector', dilatada)
    
    
    cv2.imshow('Original Video', frame1)
    
    
    # Retreive the ASCII code of the key pressed
    k = cv2.waitKey(1) & 0xff
    
    # Check if 'q' key is pressed.
    if k == ord('q'):
        
        # Break the loop:
        break

# Close the windows.q
cv2.destroyAllWindows()

Vehicle Counter:1
Vehicle Counter:2
Vehicle Counter:3
Vehicle Counter:4
Vehicle Counter:5
Vehicle Counter:6
Vehicle Counter:7
Vehicle Counter:8
Vehicle Counter:9
Vehicle Counter:10
Vehicle Counter:11
Vehicle Counter:12
